# GPS Team Project 2
## EVK-M8T 수신기의 통신을 통한 실시간 RINEX 항법파일 작성
**해당 코드는 MATLAB에서 실시간 데이터를 취득하고, PYTHON에서 가공하여 항법파일을 생성함**

### 1. gobal variable
    * sourceFilePath  : MATLAB에서 받아온 실시간 데이터의 파일 주소
    * sourceFile      : sourceFilePath의 파일 read전용 파일 디스크립터

### 2. function
    * fileOpen       : MATLAB에서 받아온 실시간 데이터의 파일을 read전용으로 열음
    * closeFile      : 데이터를 모두 읽은 후에 fileOpen에서 열은 데이터 파일을 닫음
    * action         : fileOpen을 통해 열은 데이터 파일을 가공하는 함수. 아래의 함수들을 차례대로 호출함
    * getDataIndex   : 항법파일의 요소들에 대한 정보를 가지고있는 헤더의 index를 return 함
    * makeSubFrame   : 총 224Byte로 이루어진 항법데이터을 가공하기 위해 64Byte로 이루어진 3개의 Subframe을 return함
    * decode         : 각 Subframe은 8개의 word로 이루어져있고, 정보를 얻기위해 word를 decoding함
    * subFrameDecode : 각 Subframe의 encoding을 풀어 실제적인 항법메세지에 들어갈 데이터를 취득하여 딕셔너리로 return함
    * signCorrect    : 2의 보수 표현법 데이터의 값을 결정함. 각 변수는 twoComplement 함수를 호출하여 업데이트 함
    * twoComplement  : 2의 보수 표현법 데이터들의 부호와 값을 계산하여 return함
    * fixScale       : 딕셔너리로 취득한 데이터를 항법메세지에 들어갈 단위로 맞추고, 문자열 형식으로 만들어 return함
    * divisionScale  : 데이터와 스케일 보정값을 입력받아 데이터를 보정하고 return함
    * getSVaccuracy  : URAindex로 부터 SVaccuracy를 계산하여 입력함
    * data2str       : 데이터를 입력받아 항법메세지에 들어가는 '0.XXXXXXXXXXXXD+XX'형식으로 만들어 return함
    * writePRNEpoch  : 해당 항법파일의 PRN + Eproch를 파일에 write함
    * fileWrite      : '0.XXXXXXXXXXXXD+XX'형식으로 가공된 항법메세지를 각 자리에 알맞게 파일에 write함
    
### 3. 참고자료
#### 3.1 UBX-AID-EPH
![logo](https://user-images.githubusercontent.com/48685242/71318403-c335be00-24d3-11ea-8e92-fa0f582ea943.png)
#### 3.2 GPS-EPH message structure
![logo](https://user-images.githubusercontent.com/48685242/71318332-e6ac3900-24d2-11ea-8495-cbbb5d05c234.jpg)
#### 3.3 Number Format  : 2'complement Data, unit
![logo](https://user-images.githubusercontent.com/48685242/71318209-3d187800-24d1-11ea-91ab-32640139d556.jpg)
#### 3.4 GPS Data Record Description
![logo](https://user-images.githubusercontent.com/48685242/71320602-cab79000-24f0-11ea-8ce5-438514b10f27.png)
#### 3.5 Rinex Navigation format
![logo](https://user-images.githubusercontent.com/48685242/71318492-017fad00-24d5-11ea-90a0-6de502c40f50.png)


In [ ]:
import os, sys
import math
import time
from time import sleep
from datetime import datetime

In [ ]:
class noDataExist(Exception):
    pass

In [ ]:
sourceFilePath = "./rinexData/sourceData.txt"
sourceFile = 1

In [ ]:
def fileOpen():
    global sourceFilePath
    global sourceFile
    
    if os.path.getsize(sourceFilePath) == 0: raise noDataExist()
    sourceFile = open(sourceFilePath, 'r')

In [ ]:
def closeFile():
    global sourceFilePath
    global sourceFile
    
    sourceFile.close()

In [ ]:
def action():
    out = open("./rinexData/GPS3TEAM.txt", 'a')
    
    data = sourceFile.read()
    
    closeFile()
    
    while 1:  
        # get UBX-AID-EPH message start index
        index = getDataIndex(data)
        if(index == -1): break
            
        # data exist
        # one Data length = 224, data update 
        oneData = data[index : index + 224]
        data = data[index + 224 : ]
        
        # data -> subFrame
        subFrame = makeSubFrame(oneData)
        
        # subFrame decoding -> RINEX navigation file element
        decodeData = subFrameDecode( subFrame )
        
        # 2's complement data sign correction
        decodeData = signCorrect( decodeData )
        
        # fix scale
        fixData = fixScale(decodeData)
        
        # print prn number
        writePRNEpoch(out, oneData[12:14], fixData['toc'])
        
        # file write
        fileWrite(out, fixData)
        
    out.write("-------------------------------------------------------------------------\n")
    out.close()

In [ ]:
def getDataIndex(data):    
    # header code : b5 62 
    # Class       : 0b 
    # ID          : 31
    # data Length : 68
    
    check = "b5620b3168"
    checkSize = len(check)
    dataSize = len(data)
    
    for i in range(0, dataSize):
        if dataSize - i < checkSize:
            return -1
        
        if data[i:i+checkSize] == check:
            return i
            break
    
    return -1

In [ ]:
def makeSubFrame(oneData):
    # make subframe
    # subframe length : 64 byte
    
    subBuf1 = oneData[28:92]
    subBuf2 = oneData[92:156]
    subBuf3 = oneData[156:220]
    
    subFrame = {'subFrame1' : [],'subFrame2' : [],'subFrame3' : [] }
    
    for i in range(0, 8):
        index = i * 8
        subFrame['subFrame1'].append( decode(subBuf1[index : index + 8]) )
        subFrame['subFrame2'].append( decode(subBuf2[index : index + 8]) )
        subFrame['subFrame3'].append( decode(subBuf3[index : index + 8]) )
    
    return subFrame

In [ ]:
def decode(wordData):
    # subFrame arrangement decode
    
    w1 = wordData[0:2]
    w2 = wordData[2:4]
    w3 = wordData[4:6]
    w4 = wordData[6:]  # parity bit
    
    returnData = w3 + w2 + w1
    
    return int(returnData, 16)

In [ ]:
def subFrameDecode(subFrame):
    # subFrame element decode
    
    subFrame1 = subFrame['subFrame1']
    subFrame2 = subFrame['subFrame2']
    subFrame3 = subFrame['subFrame3']
    #############################################################
    # subFrame1 decoding                                        #
    #############################################################
    weekNumber  = (subFrame1[0] & 0xFFC000) >> 14
    code        = (subFrame1[0] & 0x003000) >> 12 
    URAindex    = (subFrame1[0] & 0x000F00) >> 8
    svHealth    = (subFrame1[0] & 0x0000FC) >> 2
    IODC1       = (subFrame1[0] & 0x000003)
    
    l2pDataflag = (subFrame1[1] & 0x800000) >> 23
    
    tgd         = (subFrame1[4] & 0x0000FF)
    
    IODC2       = (subFrame1[5] & 0xFF0000) >> 16
    toc         = (subFrame1[5] & 0x00FFFF)
    
    af2         = (subFrame1[6] & 0xFF0000) >> 16
    af1         = (subFrame1[6] & 0x00FFFF)
    
    af0         = (subFrame1[7] & 0xFFFFFC) >> 2
    
    IODC        = ( IODC1 << 8 ) |  IODC2
    #############################################################
    # subFrame2 decoding                                        #
    #############################################################
    IODE1       = (subFrame2[0] & 0xFF0000) >> 16
    crs         = (subFrame2[0] & 0x00FFFF)
    
    deltaN      = (subFrame2[1] & 0xFFFF00) >> 8
    M01         = (subFrame2[1] & 0x0000FF)
    
    M02         = (subFrame2[2] & 0xFFFFFF)
    
    cuc         = (subFrame2[3] & 0xFFFF00) >> 8
    e1          = (subFrame2[3] & 0x0000FF)
    
    e2          = (subFrame2[4] & 0xFFFFFF)
    
    cus         = (subFrame2[5] & 0xFFFF00) >> 8
    rootA1      = (subFrame2[5] & 0x0000FF)
    
    rootA2      = (subFrame2[6] & 0xFFFFFF)
    
    toe         = (subFrame2[7] & 0xFFFF00) >> 8
    fitInterval = (subFrame2[7] & 0x000080) >> 7
    AODO        = (subFrame2[7] & 0x00007C) >> 2
    
    M0          = ( M01 << 24 ) | M02
    e           = (e1 << 24 ) | e2
    rootA       = (rootA1 << 24) | rootA2
    #############################################################
    # subFrame3 decoding                                        #
    #############################################################
    cic         = (subFrame3[0] & 0xFFFF00) >> 8
    omega01     = (subFrame3[0] & 0x0000FF)
    
    omega02     = (subFrame3[1] & 0xFFFFFF)
    
    cis         = (subFrame3[2] & 0xFFFF00) >> 8
    i01         = (subFrame3[2] & 0x0000FF)
    
    i02         = (subFrame3[3] & 0xFFFFFF)
    
    crc         = (subFrame3[4] & 0xFFFF00) >> 8
    w1          = (subFrame3[4] & 0x0000FF)
    
    w2          = (subFrame3[5] & 0xFFFFFF)
    
    omegaDot    = (subFrame3[6] & 0xFFFFFF)
    
    IODE2       = (subFrame3[7] & 0xFF0000) >> 16
    iDot        = (subFrame3[7] & 0x00FFFC) >> 2
    
    omega0      = ( omega01 << 24 ) | omega02
    i0          = ( i01 << 24 ) | i02
    w           = ( w1 << 24 ) | w2
    #############################################################
    # mapping Data                                              #
    #############################################################
    mapping = {}
    mapping['weekNumber']  = weekNumber
    mapping['code']        = code
    mapping['URAindex']    = URAindex
    mapping['svHealth']    = svHealth
    mapping['l2pDataflag'] = l2pDataflag
    mapping['tgd']         = tgd
    mapping['toc']         = toc
    mapping['af2']         = af2
    mapping['af1']         = af1
    mapping['af0']         = af0
    mapping['IODC']        = IODC
    mapping['IODE1']       = IODE1
    mapping['crs']         = crs
    mapping['deltaN']      = deltaN
    mapping['cuc']         = cuc
    mapping['cus']         = cus
    mapping['toe']         = toe
    mapping['fitInterval'] = fitInterval
    mapping['AODO']        = AODO
    mapping['M0']          = M0
    mapping['e']           = e
    mapping['rootA']       = rootA
    mapping['cic']         = cic
    mapping['cis']         = cis
    mapping['crc']         = crc
    mapping['omegaDot']    = omegaDot
    mapping['IODE2']       = IODE2
    mapping['iDot']        = iDot
    mapping['omega0']      = omega0
    mapping['i0']          = i0
    mapping['w']           = w
    
    return mapping

In [ ]:
def signCorrect(decodeData):
    decodeData['tgd']      = twoComplement(decodeData['tgd'], 8)
    decodeData['af2']      = twoComplement(decodeData['af2'], 8)
    decodeData['af1']      = twoComplement(decodeData['af1'], 16)
    decodeData['af0']      = twoComplement(decodeData['af0'], 22)
    decodeData['crs']      = twoComplement(decodeData['crs'], 16)
    decodeData['deltaN']   = twoComplement(decodeData['deltaN'], 16)
    decodeData['M0']       = twoComplement(decodeData['M0'], 32)
    decodeData['cuc']      = twoComplement(decodeData['cuc'], 16)
    decodeData['cus']      = twoComplement(decodeData['cus'], 16)
    decodeData['cic']      = twoComplement(decodeData['cic'], 16)
    decodeData['omega0']   = twoComplement(decodeData['omega0'], 32)
    decodeData['cis']      = twoComplement(decodeData['cis'], 16)
    decodeData['crc']      = twoComplement(decodeData['crc'], 16)
    decodeData['i0']       = twoComplement(decodeData['i0'], 32)
    decodeData['w']        = twoComplement(decodeData['w'], 32)
    decodeData['omegaDot'] = twoComplement(decodeData['omegaDot'], 24)
    decodeData['iDot']     = twoComplement(decodeData['iDot'], 14)
    
    return decodeData

In [ ]:
def twoComplement(data, bit):
    signBit = '1'
    xorBit = '1'
    for n in range(0, bit - 1):
        signBit = signBit + '0'
        xorBit = xorBit + '1'
    
    if(data & int(signBit, 2) == 0): return data
    
    # negative sign 
        
    xorBit = int(xorBit, 2)
    xorData = data ^ xorBit
    xorData = xorData + 1
    
    return -xorData

In [ ]:
def fixScale(decodeData):
    #############################################################
    # scale fix Data                                            #
    #############################################################
    decodeData['tgd']      = data2str( divisionScale(decodeData['tgd'], -31) )
    decodeData['toc']      = data2str( divisionScale(decodeData['toc'], 4) )
    decodeData['af2']      = data2str( divisionScale(decodeData['af2'], -55) )
    decodeData['af1']      = data2str( divisionScale(decodeData['af1'], -43) )
    decodeData['af0']      = data2str( divisionScale(decodeData['af0'], -31) )
    decodeData['crs']      = data2str( divisionScale(decodeData['crs'], -5) )
    decodeData['deltaN']   = data2str( divisionScale(decodeData['deltaN'], -43) * math.pi )
    decodeData['M0']       = data2str( divisionScale(decodeData['M0'], -31) * math.pi )
    decodeData['cuc']      = data2str( divisionScale(decodeData['cuc'], -29) )
    decodeData['cus']      = data2str( divisionScale(decodeData['cus'], -29) )
    decodeData['e']        = data2str( divisionScale(decodeData['e'], -33) )
    decodeData['rootA']    = data2str( divisionScale(decodeData['rootA'], -19) )
    decodeData['toe']      = data2str( divisionScale(decodeData['toe'], 4) )
    decodeData['cic']      = data2str( divisionScale(decodeData['cic'], -29) )
    decodeData['omega0']   = data2str( divisionScale(decodeData['omega0'], -31) * math.pi )
    decodeData['cis']      = data2str( divisionScale(decodeData['cis'], -29) )
    decodeData['crc']      = data2str( divisionScale(decodeData['crc'], -5) )
    decodeData['i0']       = data2str( divisionScale(decodeData['i0'], -31)  * math.pi )
    decodeData['w']        = data2str( divisionScale(decodeData['w'], -31) * math.pi )
    decodeData['omegaDot'] = data2str( divisionScale(decodeData['omegaDot'], -43) * math.pi )
    decodeData['iDot']     = data2str( divisionScale(decodeData['iDot'], -43) * math.pi )
    #############################################################
    # scale don't fix Data                                      #
    #############################################################
    decodeData['weekNumber']  = data2str(decodeData['weekNumber'])
    decodeData['code']        = data2str(decodeData['code'])
    decodeData['URAindex']    = data2str(decodeData['URAindex'])
    decodeData['svHealth']    = data2str(decodeData['svHealth'])
    decodeData['l2pDataflag'] = data2str(decodeData['l2pDataflag'])
    decodeData['IODC']        = data2str(decodeData['IODC'])
    decodeData['IODE1']       = data2str(decodeData['IODE1'])
    decodeData['fitInterval'] = data2str(decodeData['fitInterval'])
    decodeData['AODO']        = data2str(decodeData['AODO'])
    decodeData['IODE2']       = data2str(decodeData['IODE2'])
    
    decodeData['SVaccuracy'] = getSVaccuracy(decodeData['URAindex'])
    
    return decodeData

In [ ]:
def divisionScale(data, scale):
    data = float(data)
    
    if scale > 0:
        for i in range(0, scale):
            data = data * 2
    else:
        for i in range(scale, 0):
            data = data / 2
            
    return data

In [ ]:
def getSVaccuracy(URAindex):
    URAindex = URAindex[1:]
    URAindex = float(URAindex.replace('D', 'e'))
    
    if URAindex <= 6:
        SVaccuracy = math.pow(2, 1+ URAindex/2)
    else:
        SVaccuracy = math.pow(2, URAindex - 2)
        
    SVaccuracy = round(SVaccuracy, 1)
    SVaccuracy = data2str(SVaccuracy)
    
    return SVaccuracy

In [ ]:
def data2str(data):
    tmp = format(data, '1.11e')
    strData = str(tmp)
    
    if tmp[0] != '0':
        if(data < 0):
            tmp = tmp[1:]
            strData = "-0." + tmp[0] + tmp[2:]
        else:
            strData = "0." + tmp[0] + tmp[2:]
        if strData[-3:-2] == '-':
            scale = int(strData[-2:]) - 1
        else:
            scale = int(strData[-2:]) + 1
        strData = strData[:-2] + str(scale).zfill(2)

    strData = strData.replace('e', 'D')
    strData = strData.rjust(19)
    
    return strData

In [ ]:
def writePRNEpoch(fileID, prnStr, toc):
    prn = int(prnStr, 16)
    fileID.write('%2d' % prn)
    
    now = datetime.now() 
    
    toc = toc[1:]
    measureSec = int(float(toc.replace('D', 'e')))
    
    day = now.day
    
    if now.hour * 3600 + now.minute * 60 + now.second <  measureSec :
        day = day - 1
    
    hour = int(measureSec / 3600)
    minute = int((measureSec - hour * 3600) / 60)
    second = measureSec - hour * 3600 - minute * 60
    
    fileID.write(' %s %s %s %s %s %s ' % ( now.year, now.month, day, hour, minute, second ))

In [ ]:
def fileWrite(fileID, fixData):
    fileID.write(                                fixData['af0']         + fixData['af1']        + fixData['af2']         + '\n' )
    fileID.write( '  ' + fixData['IODE1']      + fixData['crs']         + fixData['deltaN']     + fixData['M0']          + '\n' )
    fileID.write( '  ' + fixData['cuc']        + fixData['e']           + fixData['cus']        + fixData['rootA']       + '\n' )
    fileID.write( '  ' + fixData['toe']        + fixData['cic']         + fixData['omega0']     + fixData['cis']         + '\n' )
    fileID.write( '  ' + fixData['i0']         + fixData['crc']         + fixData['w']          + fixData['omegaDot']    + '\n' )
    fileID.write( '  ' + fixData['iDot']       + fixData['code']        + fixData['weekNumber'] + fixData['l2pDataflag'] + '\n' )
    fileID.write( '  ' + fixData['SVaccuracy'] + fixData['svHealth']    + fixData['tgd']        + fixData['IODC']        + '\n' )
    fileID.write( '  ' + ' 0.000000000000D+00' + fixData['fitInterval'] + ' 0.000000000000D+00' + ' 0.000000000000D+00'  + '\n' )
    
    fileID.write('\n')

# main function

In [ ]:
timeSetting = 60*30

while(1):
    try:
        fileOpen()
        
        action()
        
        print("Success")
        sleep(timeSetting)
        
    except noDataExist:
        print("data not exist")
        sleep(timeSetting)
        
    except FileNotFoundError:
        print('FIle not found')
        sleep(timeSetting)
        
    except :
        print("Fail process")
        sleep(timeSetting)